# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
tf.__version__

'2.17.0'

## Part 1 - Data Preprocessing

In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="API KEY")
project = rf.workspace("Your workspace").project("project name")
version = project.version(2)
dataset = version.download("folder")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Freshness-of-fruits-2 in folder:: 100%|██████████| 8218/8218 [00:05<00:00, 1634.39it/s]


### Preprocessing the Training set

In [58]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2)
training_set = train_datagen.flow_from_directory('/content/Freshness-of-fruits-2/train',
                                                 target_size = (64, 64),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

Found 6165 images belonging to 6 classes.


### Preprocessing the Test set

In [118]:
test_datagen = ImageDataGenerator(rescale = 1./255)
valid_set = test_datagen.flow_from_directory('/content/Freshness-of-fruits-2/valid',
                                            target_size = (64, 64),
                                            batch_size = 16,
                                            class_mode = 'categorical')

Found 1223 images belonging to 6 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [60]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [61]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.


### Step 2 - Pooling

In [62]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [63]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [64]:
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [65]:
cnn.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [66]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [67]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [69]:
cnn.add(tf.keras.layers.Dropout(0.5))

### Step 5 - Output Layer

In [70]:
cnn.add(tf.keras.layers.Dense(units=6, activation='softmax'))

## Part 3 - Training the CNN

### Compiling the CNN

In [71]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [72]:
cnn.fit(x = training_set, validation_data = valid_set, epochs = 50)

Epoch 1/50
386/386 ━━━━━━━━━━━━━━━━━━━━ 77s 194ms/step - accuracy: 0.3138 - loss: 1.5866 - val_accuracy: 0.7457 - val_loss: 0.7883
Epoch 2/50
386/386 ━━━━━━━━━━━━━━━━━━━━ 72s 187ms/step - accuracy: 0.6592 - loss: 0.8674 - val_accuracy: 0.7727 - val_loss: 0.5596
Epoch 3/50
386/386 ━━━━━━━━━━━━━━━━━━━━ 73s 188ms/step - accuracy: 0.7361 - loss: 0.7283 - val_accuracy: 0.8684 - val_loss: 0.3962
Epoch 4/50
386/386 ━━━━━━━━━━━━━━━━━━━━ 73s 188ms/step - accuracy: 0.8065 - loss: 0.5772 - val_accuracy: 0.8528 - val_loss: 0.3767
Epoch 5/50
386/386 ━━━━━━━━━━━━━━━━━━━━ 74s 190ms/step - accuracy: 0.8248 - loss: 0.5410 - val_accuracy: 0.8561 - val_loss: 0.4103
Epoch 6/50
386/386 ━━━━━━━━━━━━━━━━━━━━ 73s 189ms/step - accuracy: 0.8395 - loss: 0.5082 - val_accuracy: 0.8953 - val_loss: 0.2794
Epoch 7/50
386/386 ━━━━━━━━━━━━━━━━━━━━ 82s 189ms/step - accuracy: 0.8654 - loss: 0.4207 - val_accuracy: 0.9125 - val_loss: 0.2586
Epoch 8/50
386/386 ━━━━━━━━━━━━━━━━━━━━ 74s 190ms/step - accuracy: 0.8770 - loss: 0

## Part 4 - Making a single prediction

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/path to your input', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
test_image = test_image / 255.0
result = cnn.predict(test_image)
print(result)


In [ ]:
#print(training_set.class_indices)


{'freshapples': 0, 'freshbanana': 1, 'freshoranges': 2, 'rottenapples': 3, 'rottenbanana': 4, 'rottenoranges': 5}


In [ ]:
fresh_rotten_pairs = {
    'apple': (0, 3),
    'banana': (1, 4),
    'orange': (2, 5)
}
predicted_class = np.argmax(result)
for fruit, (fresh_idx, rotten_idx) in fresh_rotten_pairs.items():
    fresh_prob = result[0][fresh_idx]
    rotten_prob = result[0][rotten_idx]
    if predicted_class == fresh_idx or predicted_class == rotten_idx:
        freshness_index = fresh_prob / (fresh_prob + rotten_prob + 1e-8)
        freshness_status = "Fresh" if predicted_class == fresh_idx else "Rotten"
        print(f"The predicted fruit is a {fruit} and it is {freshness_status}.")
        print(f"Freshness index for {fruit}: {freshness_index:.8f}")
        break